In [1]:
import cv2
import math

from ultralytics import YOLO

In [2]:
import pytesseract
import csv
from pytesseract import Output

In [3]:
import pytesseract
from pytesseract import Output

pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

model1 = YOLO("C:\\Users\\anixi\\Downloads\\best_vc.pt")
model2 = YOLO('yolo11n.pt')

# Liste di classi per ciascun modello
classNames_model1 = ["matricula"]
classNames_model2 = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# Cattura video
vid = cv2.VideoCapture("C:\\Users\\anixi\\Downloads\\C0142.MP4")

plates_images = []

while True:
    ret, img = vid.read()
    if not ret:
        break

    # Rilevamento con il primo modello
    results1 = model1(img, stream=True)
    for r in results1:
        for box in r.boxes:
            cls = int(box.cls[0])
            if cls < len(classNames_model1):
                class_name = classNames_model1[cls]
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = round(float(box.conf[0]) * 100, 2)

                #ritaglio l'immagine della targa
                plate_image = img[y1:y2, x1:x2]

                # Aggiungi l'immagine della targa alla lista
                plates_images.append(plate_image)

                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(img, f'{class_name} {confidence}%', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Rilevamento con il secondo modello
    results2 = model2(img, stream=True)
    for r in results2:
        for box in r.boxes:
            cls = int(box.cls[0])
            if cls < len(classNames_model2):
                class_name = classNames_model2[cls]
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = round(float(box.conf[0]) * 100, 2)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f'{class_name} {confidence}%', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Mostra l'immagine con le rilevazioni di entrambi i modelli
    cv2.imshow("Detections with Two Models", img)
    
    # Interruzione con ESC
    if cv2.waitKey(1) == 27:
        break

# Libera le risorse
vid.release()
cv2.destroyAllWindows()

['eng', 'osd']

0: 384x640 (no detections), 265.2ms
Speed: 17.5ms preprocess, 265.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 bus, 193.8ms
Speed: 8.1ms preprocess, 193.8ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 145.2ms
Speed: 8.0ms preprocess, 145.2ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 bus, 169.8ms
Speed: 8.0ms preprocess, 169.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 177.9ms
Speed: 0.0ms preprocess, 177.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 bus, 161.8ms
Speed: 0.0ms preprocess, 161.8ms inference, 8.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 177.7ms
Speed: 8.0ms preprocess, 177.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 bus, 194.2ms


Funzione per controllare la correttezza della targa

In [4]:
import re
plate_pattern = re.compile(r'^\d{4}\s[A-Z]{3}$')
def validate(plate):  

    if plate_pattern.match(plate):
        
        return plate
    else:
        
        plate_refactored = re.sub(r'^\s*.*?(\d{4}\s[A-Z]{3}).*$', r'\1', plate)
        return plate_refactored


In [5]:
import numpy as np
extracted_texts = []
matriculas = []

for plate_img in plates_images:

    if plate_img is not None:
    #Convierte a RGB antes de procesar
        img_rgb = cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB)
       
       
        # Preprocessing dell'immagine della targa per migliorare l'OCR 
        
        gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
        #filtered_img = cv2.GaussianBlur(gray, (5, 5), 0)
        clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
        contrast_img = clahe.apply(gray)
        resized_img = cv2.resize(contrast_img, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
        kernel = np.array([[0, -2, 0], [-2, 9, -2], [0, -2, 0]])
        sharpened_img = cv2.filter2D(resized_img, -1, kernel)

        #Texto y localización en imagen de cada palabra
        text = pytesseract.image_to_string(sharpened_img, config='--psm 8', output_type=Output.STRING)

        text = validate(text)

        if plate_pattern.match(text):
            matriculas.append(text)
            print("\nMATRÍCULA DETECTADA:", text)

        cv2.imshow('img', sharpened_img)
        cv2.waitKey(0)

   

else:
    print('Error de imagen')





MATRÍCULA DETECTADA: 3713 LKC


MATRÍCULA DETECTADA: 3713 LKC


MATRÍCULA DETECTADA: 2510 FEN


MATRÍCULA DETECTADA: 2515 FER


MATRÍCULA DETECTADA: 0053 JEW


MATRÍCULA DETECTADA: 3685 KEW


MATRÍCULA DETECTADA: 3685 KRU


MATRÍCULA DETECTADA: 3685 KEW


MATRÍCULA DETECTADA: 3585 KEW


MATRÍCULA DETECTADA: 3535 KYW


MATRÍCULA DETECTADA: 5295 ETR


MATRÍCULA DETECTADA: 5595 ETR


MATRÍCULA DETECTADA: 8555 UTE


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 1770 JYG


MATRÍCULA DETECTADA: 6098 SYM


MATRÍCULA DETECTADA: 1965 KBP


MATRÍCULA DETECTADA: 1965 KBP


MATRÍCULA DETECTADA: 1965 KSP


MATRÍCULA DETECTADA: 1965 KBP


MATRÍCULA DETECTADA: 1965 KBP


MATRÍCULA DETECTADA: 1965 KBP


MATRÍCU

In [6]:
dup = set()  # Insieme per tracciare elementi già visti

for item in matriculas:
    if item not in dup:
        print("\nMatricula: ", item)
        dup.add(item)


Matricula:  3713 LKC


Matricula:  2510 FEN


Matricula:  2515 FER


Matricula:  0053 JEW


Matricula:  3685 KEW


Matricula:  3685 KRU


Matricula:  3585 KEW


Matricula:  3535 KYW


Matricula:  5295 ETR


Matricula:  5595 ETR


Matricula:  8555 UTE


Matricula:  1770 JYG


Matricula:  6098 SYM


Matricula:  1965 KBP


Matricula:  1965 KSP


Matricula:  1855 KBP


Matricula:  1955 RRP


Matricula:  9855 KBP


Matricula:  1955 KEP

